In [ ]:
# check flagFinish

import os

path='runAux/cfgs_run1'
with open(path) as f:
    t=f.readlines()
    cfgs=[ele.strip('\n') for ele in t]
    cfgs.sort()

for cfg in cfgs:
    path='run/'+cfg+'/'
    flag=False
    if 'flagFinish' in os.listdir(path):
        flag=True

    # print(flag)
    if not flag:
        print(cfg)
        for file in os.listdir(path):
            if file.endswith('loopS.h5'):
                print(file)
                # os.remove(path+file)
            

In [ ]:


'''
cat runAux/cfgs_runt | xargs -n 1 -I @ -P 10 python3 -u run2out.py -c @ > log/run2out.out & 
'''
import os, click, h5py, re, pickle
import numpy as np

lat_L,lat_T=24,48
SorL='S'

Gs=['id','gx','gy','gz','gt','g5','g5gx','g5gy','g5gz','g5gt','sgmyz','sgmzx','sgmxy','sgmtx','sgmty','sgmtz']

# @click.command()
# @click.option('-c','--cfg')
def run(cfg):
    inpath=f'run/{cfg}/'
    outpath=f'out/{cfg}/'
    os.makedirs(outpath,exist_ok=True)

    infile=inpath+f'Diagram_{cfg[1:]}_loop{SorL}.h5'
    outfile=outpath+f'loop{SorL}.h5'
    outfile_flag=outfile+'_flag'
    if (not os.path.isfile(outfile)) or os.path.isfile(outfile_flag):
        with open(outfile_flag,'w') as f:
            pass
        with h5py.File(outfile,'w') as fw, h5py.File(infile) as f:
            st='sx00sy00sz00st00'
            ps=f[st]['PhiPhi']['mvec'][:]
        
            fw.create_dataset('moms',data=ps)
            fw.create_dataset('t,mom,G',data=[])
            fw.create_dataset('Gs',data=Gs)
            
            sign_g=np.array([1, 1,1,1,1, 1, 1,1,1,1, 1j,-1j,1j, 1j,1j,1j])
            
            for seed in f[st]['PhiPhi'].keys():
                if seed == 'mvec':
                    continue
                for id in f[st]['PhiPhi'][seed].keys():
                    t=f[st]['PhiPhi'][seed][id]['up'][:]
                    t=t[...,0]+1j*t[...,1]
                    fw.create_dataset('data/'+seed+'/'+id+'/up',data=t)
                    
        os.remove(outfile_flag)
        
    print('flag_cfg_done: '+cfg)
    
run('a0000')

# check with previous

In [ ]:
import h5py,os,re,pickle
import numpy as np

path='/project/s1174/lyan/code/scratch/run/NST_f/cA211.53.24/loop/run/a0026/Diagram_0026_loopL.h5'
path0='/project/s1174/lyan/code/scratch/run/NST_d/cA211.53.24/loops/run/a0026/Diagram0026_insertLoop.h5'

with h5py.File(path) as f, h5py.File(path0) as f0:
    src='sx00sy00sz00st00'
    
    t=3; gm=5; id='3'
    mom=2; mom0=15
    print(f[src]['PhiPhi']['mvec'][mom],f0[src]['mvec'][mom0])
    print(-f[src]['PhiPhi']['seed=423464188'][f'id={id}']['up'][t,mom,gm])
    print(f0[src][f'stoc423464188id{id}_1']['up'][t,mom0,gm])

In [ ]:
import h5py,os,re,pickle
import numpy as np

path='/project/s1174/lyan/code/scratch/run/NST_f/cA211.53.24/loop/run/a0026/Diagram_0026_loopS.h5'
path0='/project/s1174/lyan/code/scratch/run/NST_d/cA211.53.24/loops/data_out/a0026/Diagram0026_pi0Loop.h5'

with h5py.File(path) as f, h5py.File(path0) as f0:
    src='sx00sy00sz00st00'
    
    t=3; gm=5; id='3'
    mom=2; mom0=12
    print(f[src]['PhiPhi']['mvec'][mom],f0[src]['mvec'][mom0])
    print(-f[src]['PhiPhi']['seed=2088513427'][f'id={id}']['up'][t,mom,gm])
    print(f0[src][f'stoc2088513427id{id}_1']['up'][t,mom0,0])

In [ ]:
import h5py,os,re,pickle
import numpy as np

lat_L,lat_T=24,48

gjList=['id','gx','gy','gz','gt','g5','g5gx','g5gy','g5gz','g5gt','sgmyz','sgmzx','sgmxy','sgmtx','sgmty','sgmtz']
gtCj={'id':1,'gx':-1,'gy':-1,'gz':-1,'gt':1,'g5':-1,'g5gx':-1,'g5gy':-1,'g5gz':-1,'g5gt':1,'sgmxy':-1,'sgmyz':-1,'sgmzx':-1,'sgmtx':1,'sgmty':1,'sgmtz':1}

def run(cfg):
    inpath=f'run/{cfg}/'
    outpath=f'out/{cfg}/'
    os.makedirs(outpath,exist_ok=True)

    infile=inpath+f'Diagram_{cfg[1:]}_jPP.h5'
    outfile=outpath+'jPP.h5'
    with h5py.File(outfile,'w') as fw, h5py.File(infile) as f:
        st='sx00sy00sz00st00'
        pfs=f[st]['PhiPhi']['mvec'][:]
        pis=[[int(i) for i in pi[3:].split('_')] for pi in f[st]['PhiPhi'].keys() if pi!='mvec']
        pis.sort()
        str_pis=['pi='+'_'.join([str(i) for i in pi]) for pi in pis]
        moms_new=[list(pi)+list(pf) for pi in pis for pf in pfs]
        moms_new.sort()
        
        dic={}
        for i,pi in enumerate(pis):
            dic[tuple(pi)]=i
        map_pi=[dic[tuple(mom[:3])] for mom in moms_new]
        dic={}
        for i,pf in enumerate(pfs):
            dic[tuple(pf)]=i
        map_pf=[dic[tuple(mom[3:])] for mom in moms_new]
        
        fw.create_dataset('moms',data=moms_new)
        fw.create_dataset('tf,mom,Gf,Gi',data=[])
        
        
        sign_gi=np.array([1,1,1]) # id,g5,g5gt
        sign_gf=np.array([1, 1,1,1,1, 1, 1,1,1,1, 1j,-1j,1j, 1j,1j,1j])
        sign_gi*= [gtCj[gm] for gm in ['id','g5','g5gt']]
        gis=['i_gi={}'.format(i) for i in range(len(sign_gi))]
        
        for i_st in range(lat_T):
            st='sx00sy00sz00st{:02d}'.format(i_st)
            st_new='st{:03d}'.format(i_st)
            for fl in ['uu','ud','du','dd']:
                fla='q1q2='+fl
                for sme in ['LL','SL','SS']:
                    smear='sisf='+sme
                    opab=fl[0]+'B'+fl[1]+sme[1]+'_'+fl[0]+'B'+fl[1]+sme[0]
                    
                    t=np.array([[f[st]['PhiPhi'][pi][fla][smear][gi][:] for pi in str_pis] for gi in gis])
                    t=t[...,0]+1j*t[...,1]
                    t=t[:,map_pi,:,map_pf]
                    t=np.transpose(t,[2,0,3,1]) # tf,mom,Gf,Gi
                    t*=-1 # (-1) for the (-1) in front of PhiPhi
                    t*=sign_gi[None,None,None,:]
                    t*=sign_gf[None,None,:,None]
                    fw.create_dataset('data/'+st_new+'/'+opab,data=t.astype('complex64'))
                    print(i_st,fla,smear,end='          \r')
                    
cfg='a0026'
run(cfg)

In [ ]:
import h5py,os,re,pickle
import numpy as np

path0='/project/s1174/lyan/code/projectData/NST_d/cA211.530.24/data_post/a0026/P.h5_jPP-b-tensor'
path='/project/s1174/lyan/code/scratch/run/NST_f/cA211.53.24/jPP/out/a0026/jPP.h5'
with h5py.File(path) as f, h5py.File(path0) as f0:
    st='st000'
    tf=47; mom0=25; mom=23; 
    print(f0['moms'][mom0])
    print(f['moms'][mom])
    print(f['data'][st]['uBdS_uBdS'][tf,mom,5,1])
    print(f0['data'][st]['pi-_pi-'][tf,mom0])
    
    print((f['data'][st]['uBuS_uBuS'][tf,mom,5,1]+f['data'][st]['dBdS_dBdS'][tf,mom,5,1])/2)
    print(f0['data'][st]['pi0_pi0'][tf,mom0])
    
    
path0='/project/s1174/lyan/code/projectData/NST_d/cA211.530.24/data_post/a0026/jPi.h5_jPP-b-tensor'
path='/project/s1174/lyan/code/scratch/run/NST_f/cA211.53.24/jPP/out/a0026/jPP.h5'
with h5py.File(path) as f, h5py.File(path0) as f0:
    st='st000'
    tf=47; mom0=0; mom=0; gm=15
    print(f['moms'][mom])
    tu=f['data'][st]['uBuL_uBuS'][tf,mom,gm,1] *(-1j)/np.sqrt(2)
    td=f['data'][st]['dBdL_dBdS'][tf,mom,gm,1] *(-1j)/np.sqrt(2)
    print(tu-td)
    print(f0['data'][st]['j+_pi0'][tf,mom0,gm])
    
    print(tu+td)
    print(f0['data'][st]['j-_pi0'][tf,mom0,gm])
    
    
path='/project/s1174/lyan/code/scratch/run/NST_f/cA211.53.24/jPP/out/a0026/jPP.h5'
with h5py.File(path) as f:
    st='st000'
    tf=47; mom1=23; mom2=25; Gf=5; Gi=1
    print(f['moms'][mom1],f['moms'][mom2])
    t1=f['data'][st]['uBuL_uBuS'][tf,mom1,Gf,Gi]
    t2=np.conj(f['data'][st]['dBdL_dBdS'][tf,mom2,Gf,Gi])
    print(t1)
    print(t2)
    
    st='st000'
    tf=47; mom1=23; mom2=25; Gf=5; Gi=1
    print(f['moms'][mom1],f['moms'][mom2])
    t1=f['data'][st]['uBdL_uBdL'][tf,mom1,Gf,Gi]
    t2=np.conj(f['data'][st]['uBdL_uBdL'][tf,mom2,Gf,Gi])
    print(t1)
    print(t2)